# Cleaning Data

In [8]:
import pandas as pd
import numpy as np

Load Dataset

In [9]:
player = 'cole'
data = pd.read_csv('./data/' + player + '.csv')
print(data)

       Unnamed: 0 pitch_type   game_date  release_speed  release_pos_x  \
0               0         KC  2018-10-14           80.6          -2.04   
1               1         SL  2018-10-14           88.0          -2.01   
2               2         CH  2018-10-14           86.1          -2.23   
3               3         CH  2018-10-14           87.5          -2.14   
4               4         KC  2018-10-14           80.5          -2.04   
...           ...        ...         ...            ...            ...   
21489        5407         SL  2019-03-29           89.8          -2.35   
21490        5408         FF  2019-03-29           96.3          -2.16   
21491        5409         SL  2019-03-29           88.5          -2.34   
21492        5410         FT  2019-03-29           96.7          -2.34   
21493        5411         FF  2019-03-29           96.2          -2.24   

       release_pos_z   player_name  batter  pitcher     events  ... fld_score  \
0               5.61  Cole, Ge

/Users/jackmaginnes/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Simplify the batters on base columns:

In [3]:
data['on_base'] = np.sum([
    pd.notnull(data['on_1b']),
    pd.notnull(data['on_2b']),
    pd.notnull(data['on_3b'])
], axis=0)



In [4]:
to_shift_down = [
    'events',
    'zone',
    'type',
    'bb_type',
    'plate_x',
    'plate_z'
]

groups = data.groupby(['game_pk', 'inning'])

list_of_dfs = []
for _, g in groups:
    list_of_dfs.append(g)


for i, df in enumerate(list_of_dfs):
    for column in df:
        if column in to_shift_down:
            df["prev_" + column] = df[column].shift(periods=1)
    list_of_dfs[i] = df
print(list_of_dfs[0])
            
    
            
cleaned_data = pd.concat(list_of_dfs, axis=0, sort=False)
    
# size = len(data[column])
# print(size)

# for column in data:
#     if column in to_shift_down:
#         data[column].shift(1)
                
# for column in data:
#     if column in to_shift_down:
#         for i, row in enumerate(data[column]):
#             if i != 0 and i < size - 2:
#                 if (data['game_pk'][i] != data['game_pk'][i-1]) or (data['inning'][i] != data['inning'][i-1]):
#                     data.loc[column, i] = np.NaN

       Unnamed: 0 pitch_type   game_date  release_speed  release_pos_x  \
16064       16064         FF  2013-06-11           97.7          -2.36   
16065       16065         FF  2013-06-11           99.6          -2.29   
16066       16066         SI  2013-06-11           97.4          -2.30   
16067       16067         FF  2013-06-11           97.9          -2.24   
16068       16068         FF  2013-06-11           98.7          -2.23   
16069       16069         FF  2013-06-11           97.4          -2.51   
16070       16070         SL  2013-06-11           86.0          -2.53   
16071       16071         SI  2013-06-11           97.6          -2.56   
16072       16072         FF  2013-06-11           97.2          -2.36   
16073       16073         CH  2013-06-11           87.5          -2.47   
16074       16074         FF  2013-06-11           96.4          -2.23   
16075       16075         FF  2013-06-11           96.5          -2.30   
16076       16076         FF  2013-06-

Now, we explicitly list the columns that we want to keep:

In [5]:
keep_columns = [
    'pitch_type',
    'game_date',
    'release_speed',
    'batter',
    'stand',
    'balls',
    'strikes',
    'plate_x',
    'plate_y',
    'plate_z',
    'outs_when_up',
    'inning',
    'game_pk',
    'pitch_num',
    'bat_score',
    'fld_score',
    'on_base',
    'prev_events',
    'prev_zone',
    'prev_type',
    'prev_bb_type',
    'prev_plate_x',
    'prev_plate_z'
]

Delete the unnecessary columns:

In [6]:
for column in cleaned_data:
    if column not in keep_columns:
        cleaned_data.pop(column)

print("Kept Columns:")
for column in data:
    print(column)

Kept Columns:
Unnamed: 0
pitch_type
game_date
release_speed
release_pos_x
release_pos_z
player_name
batter
pitcher
events
description
spin_dir
spin_rate_deprecated
break_angle_deprecated
break_length_deprecated
zone
des
game_type
stand
p_throws
home_team
away_team
type
hit_location
bb_type
balls
strikes
game_year
pfx_x
pfx_z
plate_x
plate_z
on_3b
on_2b
on_1b
outs_when_up
inning
inning_topbot
hc_x
hc_y
tfs_deprecated
tfs_zulu_deprecated
fielder_2
umpire
sv_id
vx0
vy0
vz0
ax
ay
az
sz_top
sz_bot
hit_distance_sc
launch_speed
launch_angle
effective_speed
release_spin_rate
release_extension
game_pk
pitcher.1
fielder_2.1
fielder_3
fielder_4
fielder_5
fielder_6
fielder_7
fielder_8
fielder_9
release_pos_y
estimated_ba_using_speedangle
estimated_woba_using_speedangle
woba_value
woba_denom
babip_value
iso_value
launch_speed_angle
at_bat_number
pitch_number
pitch_name
home_score
away_score
bat_score
fld_score
post_away_score
post_home_score
post_bat_score
post_fld_score
if_fielding_alignment
of_fi

In the data we have, the pitch (label) and the results of the pitch are in the same row. We would like the label to be on the same row as the previous pitch data, so we need to shift some of the columns down one row. 

Save the data:

In [7]:
cleaned_data.to_csv('./cleaned_data/' + player + '.csv')